In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.sql import text
import requests
import discogs_client
import psycopg2
from pprint import pprint
import time

from discogs_client import Client
DB_NAME = "Music Record Labels"
DB_USER = "postgres"
DB_PASS = 'S_1pLa2k'
DB_HOST = "localhost"
DB_PORT = "5432"

# engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}")
# print(engine)
# conn = engine.connect()
# print(conn)
# url = 'https://geocode.xyz/'
# api_key = 'lHUvhuTIhLQxtKGIIIWrwjCCJyQytXauPPMcTYCy'
# Current token: lHUvhuTIhLQxtKGIIIWrwjCCJyQytXauPPMcTYCy


df = pd.read_xml(r"C:\Users\micha\PycharmProjects\DE classwork\Lab-Music-Record-Label\discogs_20230101_labels.xml")
display(df)

,images,id,name,contactinfo,profile,data_quality,urls,sublabels,parentLabel
0,NaN,1,Planet E,Planet E Communications\r\nP.O. Box 27218\r\nD...,[a=Carl Craig]'s classic techno label founded ...,Needs Vote,NaN,NaN,None
1,NaN,2,Earthtones Recordings,Seasons Recordings\r\n2236 Pacific Avenue\r\nS...,California deep house label founded by [a=Jami...,Correct,NaN,NaN,None
2,NaN,3,Seasons Recordings,"Seasons Recordings\r\nCosta Mesa, CA 92627\r\n...",California deep-house label founded by [a=Jami...,Needs Vote,NaN,NaN,None
3,NaN,4,Siesta Music,Siesta Records\r\n1913 Via Encantadoras\r\nSan...,None,Needs Vote,NaN,NaN,None
4,NaN,5,Svek,Svek office \r\nStephan Grieder \r\nFax: +46 (...,None,Correct,NaN,NaN,Goldhead Music
...,...,...,...,...,...,...,...,...,...
1946241,NaN,3078049,5th Gear Productions,None,None,Needs Major Changes,NaN,NaN,None
1946242,NaN,3078052,"Tunnell, Jimi Music",None,None,Needs Major Changes,NaN,NaN,None
1946243,NaN,3078055,Warrior Gospel Records,None,None,Needs Major Changes,NaN,NaN,None
1946244,NaN,3078058,Allegretto Music,None,None,Needs Major Changes,NaN,NaN,None


In [2]:
del df['images']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1946246 entries, 0 to 1946245
Data columns (total 8 columns):
 #   Column        Dtype  
---  ------        -----  
 0   id            int64  
 1   name          object 
 2   contactinfo   object 
 3   profile       object 
 4   data_quality  object 
 5   urls          float64
 6   sublabels     float64
 7   parentLabel   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 118.8+ MB


In [3]:
df = df.drop('urls', axis=1)
df = df.drop('sublabels',axis=1)

In [4]:
df['data_quality'].value_counts()

data_quality
Needs Major Changes     1333815
Needs Vote               474834
Correct                  132960
Complete and Correct       4633
Needs Minor Changes           3
Entirely Incorrect            1
Name: count, dtype: int64

In [5]:
df = df[df['data_quality'] == 'Complete and Correct']

df

,id,name,contactinfo,profile,data_quality,parentLabel
725,774,Hed Kandi Records,"MSHK Limited\r\n103 Gaunt Street\r\nLondon, SE...",Record company of the UK-based house label [l6...,Complete and Correct,Ministry Of Sound
1338,1450,Dot Com,None,Label owned by [a=Phil Wells (2)] from [a=Subh...,Complete and Correct,None
1598,1730,Test Blue,None,"Label from the Valve crew, Dillinja & Lemon D",Complete and Correct,Test Recordings
2140,2323,Warner Music (Australia),None,[b]DO NOT USE.[/b]\r\nThis is a legacy label. ...,Complete and Correct,None
2266,2468,Lifetime Music,None,Founded in summer 2000 by [a=Beroshima] & Jack...,Complete and Correct,None
...,...,...,...,...,...,...
1873690,2847239,Musicando Production And Promotion,None,Affiliated to [a=Francesco Digilio].,Complete and Correct,None
1879123,2864591,министерство культуры CCCP,None,[b]DO NOT USE.[/b],Complete and Correct,None
1882605,2875724,Doppelkirche Bonn-Schwarzrheindorf,Doppelkirche St. Maria und St. Clemens\r\nDixs...,Catholic parish church in Romanesque style fro...,Complete and Correct,None
1895544,2917031,Rocke Grace Music,None,None,Complete and Correct,None


In [6]:
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df

,id,name,contactinfo,profile,data_quality,parentLabel
725,774,Hed Kandi Records,"MSHK Limited\r\n103 Gaunt Street\r\nLondon, SE...",Record company of the UK-based house label [l6...,Complete and Correct,Ministry Of Sound
2878,3141,Concept,Concept Records\r\n24A Barry Street\r\nBradfor...,Trance label from the UK. The label becomes [l...,Complete and Correct,Global Beat Records
3505,3824,Fuse,Fuse\r\nbkb Dance Department gmbh\r\nAuhofstra...,German Trance label\r\nDistributed by Music Ma...,Complete and Correct,Massive Records
5824,6399,Sheffield Tunes,Sheffield Tunes GbR\r\nNeumühlen 17 \r\n22763 ...,Founded in 2000 by the group Scooter: H.P. Bax...,Complete and Correct,Kontor Records GmbH
5863,6445,Thrust (2),46 Cours Julien\r\n13006 Marseille France\r\nP...,"Electro, techno label from Marseille, south of...",Complete and Correct,B.E.A.S.T. Records
...,...,...,...,...,...,...
1217394,1526929,Fr3ak3rz Rec.,fr3ak3rz@hotmail.com,"Electronic music label based in Amsterdam, Net...",Complete and Correct,Co2 Recordings
1309793,1630287,Believe Digital GmbH,Van-der-Smissen-Str. 4\r\n22767 Hamburg\r\nGer...,German branch of European music distributor [l...,Complete and Correct,Believe Digital
1469533,1806978,"Beethoven - The Complete Symphonies (Kempe, WRC)",World Record Club: Registered Office - 605 Cam...,World Record Club series of quadrophonic LPs f...,Complete and Correct,World Record Club
1523918,1867128,Nota Bene (8),Nota Bene d.o.o.\r\nDraškovićeva 23\r\n10000 Z...,We are independent music label from Croatia of...,Complete and Correct,NOTA BENE RECORDS


In [7]:
user_token = 'lHUvhuTIhLQxtKGIIIWrwjCCJyQytXauPPMcTYCy'

# Define a function to fetch the number of releases for a given label ID with a delay
def get_num_of_releases_with_delay(label_id):
    url = f'https://api.discogs.com/labels/{label_id}/releases'
    headers = {'Authorization': f'Discogs token={user_token}'}
    response = requests.get(url, headers=headers)
    pprint(response.json())
    time.sleep(.5)  # Add a delay of 1 second between requests
    if response.status_code == 200:
        return len(response.json().get('releases', []))
    return 0

# Apply the function to each label ID and store the results in a new column
df['num_of_releases'] = df['id'].apply(get_num_of_releases_with_delay)

# Display the DataFrame
display(df)

{'pagination': {'items': 646,
                'page': 1,
                'pages': 13,
                'per_page': 50,
                'urls': {'last': 'https://api.discogs.com/labels/774/releases?page=13&per_page=50',
                         'next': 'https://api.discogs.com/labels/774/releases?page=2&per_page=50'}},
 'releases': [{'artist': '45Dip*',
               'catno': 'HEDK005',
               'format': 'CD, Album',
               'id': 16067,
               'resource_url': 'https://api.discogs.com/releases/16067',
               'stats': {'community': {'in_collection': 202, 'in_wantlist': 56},
                         'user': {'in_collection': 0, 'in_wantlist': 0}},
               'status': 'Accepted',
               'thumb': 'https://i.discogs.com/gYc1KscKB6vXJ-3VWuM8RtbIB_x-swQZElAlbHtj9q8/rs:fit/g:sm/q:40/h:150/w:150/czM6Ly9kaXNjb2dz/LWRhdGFiYXNlLWlt/YWdlcy9SLTE2MDY3/LTEyNTc4ODI2ODAu/anBlZw.jpeg',
               'title': 'The Acid Lounge',
               'year': 2000},
     

,id,name,contactinfo,profile,data_quality,parentLabel,num_of_releases
725,774,Hed Kandi Records,"MSHK Limited\r\n103 Gaunt Street\r\nLondon, SE...",Record company of the UK-based house label [l6...,Complete and Correct,Ministry Of Sound,50
2878,3141,Concept,Concept Records\r\n24A Barry Street\r\nBradfor...,Trance label from the UK. The label becomes [l...,Complete and Correct,Global Beat Records,4
3505,3824,Fuse,Fuse\r\nbkb Dance Department gmbh\r\nAuhofstra...,German Trance label\r\nDistributed by Music Ma...,Complete and Correct,Massive Records,32
5824,6399,Sheffield Tunes,Sheffield Tunes GbR\r\nNeumühlen 17 \r\n22763 ...,Founded in 2000 by the group Scooter: H.P. Bax...,Complete and Correct,Kontor Records GmbH,50
5863,6445,Thrust (2),46 Cours Julien\r\n13006 Marseille France\r\nP...,"Electro, techno label from Marseille, south of...",Complete and Correct,B.E.A.S.T. Records,42
...,...,...,...,...,...,...,...
1217394,1526929,Fr3ak3rz Rec.,fr3ak3rz@hotmail.com,"Electronic music label based in Amsterdam, Net...",Complete and Correct,Co2 Recordings,2
1309793,1630287,Believe Digital GmbH,Van-der-Smissen-Str. 4\r\n22767 Hamburg\r\nGer...,German branch of European music distributor [l...,Complete and Correct,Believe Digital,3
1469533,1806978,"Beethoven - The Complete Symphonies (Kempe, WRC)",World Record Club: Registered Office - 605 Cam...,World Record Club series of quadrophonic LPs f...,Complete and Correct,World Record Club,31
1523918,1867128,Nota Bene (8),Nota Bene d.o.o.\r\nDraškovićeva 23\r\n10000 Z...,We are independent music label from Croatia of...,Complete and Correct,NOTA BENE RECORDS,9


In [8]:
user_token = 'lHUvhuTIhLQxtKGIIIWrwjCCJyQytXauPPMcTYCy'

# Define a function to fetch the earliest and latest release years for a given label ID
def get_min_max_release_years(label_id):
    url = f'https://api.discogs.com/labels/{label_id}/releases'
    headers = {'Authorization': f'Discogs token={user_token}'}
    response = requests.get(url, headers=headers)
    time.sleep(1)  # Add a delay of 1 second between requests
    if response.status_code == 200:
        releases = response.json().get('releases', [])
        release_years = [int(release.get('year')) for release in releases if release.get('year')]
        if release_years:
            return min(release_years), max(release_years)
    return None, None

# Apply the function to each label ID and store the results in new columns
df['min_release_year'], df['max_release_year'] = zip(*df['id'].apply(get_min_max_release_years))

# Display the DataFrame
display(df)


,id,name,contactinfo,profile,data_quality,parentLabel,num_of_releases,min_release_year,max_release_year
725,774,Hed Kandi Records,"MSHK Limited\r\n103 Gaunt Street\r\nLondon, SE...",Record company of the UK-based house label [l6...,Complete and Correct,Ministry Of Sound,50,1999,2002
2878,3141,Concept,Concept Records\r\n24A Barry Street\r\nBradfor...,Trance label from the UK. The label becomes [l...,Complete and Correct,Global Beat Records,4,1996,1996
3505,3824,Fuse,Fuse\r\nbkb Dance Department gmbh\r\nAuhofstra...,German Trance label\r\nDistributed by Music Ma...,Complete and Correct,Massive Records,32,1997,2001
5824,6399,Sheffield Tunes,Sheffield Tunes GbR\r\nNeumühlen 17 \r\n22763 ...,Founded in 2000 by the group Scooter: H.P. Bax...,Complete and Correct,Kontor Records GmbH,50,2000,2003
5863,6445,Thrust (2),46 Cours Julien\r\n13006 Marseille France\r\nP...,"Electro, techno label from Marseille, south of...",Complete and Correct,B.E.A.S.T. Records,42,1996,2003
...,...,...,...,...,...,...,...,...,...
1217394,1526929,Fr3ak3rz Rec.,fr3ak3rz@hotmail.com,"Electronic music label based in Amsterdam, Net...",Complete and Correct,Co2 Recordings,2,2018,2018
1309793,1630287,Believe Digital GmbH,Van-der-Smissen-Str. 4\r\n22767 Hamburg\r\nGer...,German branch of European music distributor [l...,Complete and Correct,Believe Digital,3,2019,2019
1469533,1806978,"Beethoven - The Complete Symphonies (Kempe, WRC)",World Record Club: Registered Office - 605 Cam...,World Record Club series of quadrophonic LPs f...,Complete and Correct,World Record Club,31,1975,1979
1523918,1867128,Nota Bene (8),Nota Bene d.o.o.\r\nDraškovićeva 23\r\n10000 Z...,We are independent music label from Croatia of...,Complete and Correct,NOTA BENE RECORDS,9,2018,2021


In [9]:
user_token = 'lHUvhuTIhLQxtKGIIIWrwjCCJyQytXauPPMcTYCy'

# Define a function to fetch the earliest release year for a given label ID
def get_earliest_release_year(label_id):
    url = f'https://api.discogs.com/labels/{label_id}/releases'
    headers = {'Authorization': f'Discogs token={user_token}'}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        releases = response.json().get('releases', [])
        release_years = [float(release.get('year')) for release in releases if release.get('year')]
        if release_years:
            return float(min(release_years))
    return None

# Apply the function to each label ID and store the results in a new column
df['min_release_year'] = df['id'].apply(get_earliest_release_year)

# Display the DataFrame
display(df)


,id,name,contactinfo,profile,data_quality,parentLabel,num_of_releases,min_release_year,max_release_year
725,774,Hed Kandi Records,"MSHK Limited\r\n103 Gaunt Street\r\nLondon, SE...",Record company of the UK-based house label [l6...,Complete and Correct,Ministry Of Sound,50,1999.0,2002
2878,3141,Concept,Concept Records\r\n24A Barry Street\r\nBradfor...,Trance label from the UK. The label becomes [l...,Complete and Correct,Global Beat Records,4,1996.0,1996
3505,3824,Fuse,Fuse\r\nbkb Dance Department gmbh\r\nAuhofstra...,German Trance label\r\nDistributed by Music Ma...,Complete and Correct,Massive Records,32,1997.0,2001
5824,6399,Sheffield Tunes,Sheffield Tunes GbR\r\nNeumühlen 17 \r\n22763 ...,Founded in 2000 by the group Scooter: H.P. Bax...,Complete and Correct,Kontor Records GmbH,50,2000.0,2003
5863,6445,Thrust (2),46 Cours Julien\r\n13006 Marseille France\r\nP...,"Electro, techno label from Marseille, south of...",Complete and Correct,B.E.A.S.T. Records,42,1996.0,2003
...,...,...,...,...,...,...,...,...,...
1217394,1526929,Fr3ak3rz Rec.,fr3ak3rz@hotmail.com,"Electronic music label based in Amsterdam, Net...",Complete and Correct,Co2 Recordings,2,NaN,2018
1309793,1630287,Believe Digital GmbH,Van-der-Smissen-Str. 4\r\n22767 Hamburg\r\nGer...,German branch of European music distributor [l...,Complete and Correct,Believe Digital,3,NaN,2019
1469533,1806978,"Beethoven - The Complete Symphonies (Kempe, WRC)",World Record Club: Registered Office - 605 Cam...,World Record Club series of quadrophonic LPs f...,Complete and Correct,World Record Club,31,NaN,1979
1523918,1867128,Nota Bene (8),Nota Bene d.o.o.\r\nDraškovićeva 23\r\n10000 Z...,We are independent music label from Croatia of...,Complete and Correct,NOTA BENE RECORDS,9,NaN,2021


In [10]:
user_token = 'lHUvhuTIhLQxtKGIIIWrwjCCJyQytXauPPMcTYCy'

# Define a function to fetch the earliest release year for a given label ID
def get_max_release_year(label_id):
    url = f'https://api.discogs.com/labels/{label_id}/releases'
    headers = {'Authorization': f'Discogs token={user_token}'}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        releases = response.json().get('releases', [])
        release_years = [int(release.get('year')) for release in releases if release.get('year')]
        if release_years:
            return str(max(release_years)) # Cast to float
    return None

# Apply the function to each label ID and store the results in a new column
df['max_release_year'] = df['id'].apply(get_max_release_year)

# Display the data types of the 'max_release_year' column
print("Data types in 'max_release_year' column:")
print(df['max_release_year'].dtype)

# Display the DataFrame
display(df)


Data types in 'max_release_year' column:
object


,id,name,contactinfo,profile,data_quality,parentLabel,num_of_releases,min_release_year,max_release_year
725,774,Hed Kandi Records,"MSHK Limited\r\n103 Gaunt Street\r\nLondon, SE...",Record company of the UK-based house label [l6...,Complete and Correct,Ministry Of Sound,50,1999.0,None
2878,3141,Concept,Concept Records\r\n24A Barry Street\r\nBradfor...,Trance label from the UK. The label becomes [l...,Complete and Correct,Global Beat Records,4,1996.0,None
3505,3824,Fuse,Fuse\r\nbkb Dance Department gmbh\r\nAuhofstra...,German Trance label\r\nDistributed by Music Ma...,Complete and Correct,Massive Records,32,1997.0,None
5824,6399,Sheffield Tunes,Sheffield Tunes GbR\r\nNeumühlen 17 \r\n22763 ...,Founded in 2000 by the group Scooter: H.P. Bax...,Complete and Correct,Kontor Records GmbH,50,2000.0,None
5863,6445,Thrust (2),46 Cours Julien\r\n13006 Marseille France\r\nP...,"Electro, techno label from Marseille, south of...",Complete and Correct,B.E.A.S.T. Records,42,1996.0,None
...,...,...,...,...,...,...,...,...,...
1217394,1526929,Fr3ak3rz Rec.,fr3ak3rz@hotmail.com,"Electronic music label based in Amsterdam, Net...",Complete and Correct,Co2 Recordings,2,NaN,None
1309793,1630287,Believe Digital GmbH,Van-der-Smissen-Str. 4\r\n22767 Hamburg\r\nGer...,German branch of European music distributor [l...,Complete and Correct,Believe Digital,3,NaN,None
1469533,1806978,"Beethoven - The Complete Symphonies (Kempe, WRC)",World Record Club: Registered Office - 605 Cam...,World Record Club series of quadrophonic LPs f...,Complete and Correct,World Record Club,31,NaN,None
1523918,1867128,Nota Bene (8),Nota Bene d.o.o.\r\nDraškovićeva 23\r\n10000 Z...,We are independent music label from Croatia of...,Complete and Correct,NOTA BENE RECORDS,9,NaN,None


In [11]:
df['name'] = df['name'].astype(str)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 90 entries, 725 to 1534751
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                90 non-null     int64  
 1   name              90 non-null     object 
 2   contactinfo       90 non-null     object 
 3   profile           90 non-null     object 
 4   data_quality      90 non-null     object 
 5   parentLabel       90 non-null     object 
 6   num_of_releases   90 non-null     int64  
 7   min_release_year  25 non-null     float64
 8   max_release_year  0 non-null      object 
dtypes: float64(1), int64(2), object(6)
memory usage: 7.0+ KB


In [13]:
# df['min_release_year'] = df['min_release_year'].astype(int)